# 🚀 Projet Final : Analyse Statistique Complète de Données Financières

## 🎯 Objectif du Projet

Réaliser une **analyse statistique professionnelle** complète sur des données financières réelles, en appliquant tous les concepts appris dans ce chapitre.

## 📋 Cahier des Charges

Vous allez analyser les rendements de **5 actions** sur une période de **2 ans** (≈500 jours de trading).

### Votre mission :

1. **📊 Analyse Descriptive Complète**
   - Statistiques de base (moyenne, médiane, écart-type)
   - Distribution des rendements
   - Détection d'anomalies

2. **📈 Visualisations Professionnelles**
   - Dashboard statistique multi-panels
   - Heatmap de corrélations
   - Séries temporelles

3. **🧪 Tests Statistiques**
   - Test de normalité
   - Test de moyenne (rendement > 0?)
   - Comparaison de performances

4. **📏 Intervalles de Confiance**
   - IC pour rendements moyens
   - IC pour volatilité

5. **📈 Modèle CAPM (Régression)**
   - Calcul des β (betas)
   - Calcul des α (alphas)
   - Validation du modèle

6. **📑 Rapport Final**
   - Synthèse des résultats
   - Recommandations d'investissement

---

## 💼 Contexte Professionnel

Vous êtes **Data Analyst** dans un fonds d'investissement. Votre manager vous demande d'analyser 5 actions pour déterminer lesquelles inclure dans un nouveau portefeuille.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, t, chi2
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Environnement configuré")

---

## 📊 Étape 1 : Génération et Chargement des Données

### 1.1 Création d'un Dataset Réaliste

Nous allons créer des données synthétiques mais réalistes pour 5 actions :

- **TECH_A** : Action technologique volatile (β=1.3)
- **FINANCE_B** : Action financière stable (β=1.0)
- **HEALTH_C** : Action santé défensive (β=0.7)
- **ENERGY_D** : Action énergétique cyclique (β=1.5)
- **CONSUMER_E** : Action consommation (β=0.9)

In [ ]:
# Paramètres de simulation
np.random.seed(42)
n_jours = 504  # ~2 ans de trading (252 jours/an)

# Génération du marché (indice de référence)
rendements_marche = np.random.normal(0.0008, 0.012, n_jours)  # 0.08% par jour, σ=1.2%

# Génération des actions selon le modèle CAPM : R = α + β*R_marché + ε
actions = {
    'TECH_A': {
        'alpha': 0.0003,  # +0.03% alpha par jour
        'beta': 1.3,
        'epsilon_std': 0.008
    },
    'FINANCE_B': {
        'alpha': 0.0001,
        'beta': 1.0,
        'epsilon_std': 0.006
    },
    'HEALTH_C': {
        'alpha': 0.0002,
        'beta': 0.7,
        'epsilon_std': 0.005
    },
    'ENERGY_D': {
        'alpha': -0.0001,  # Alpha négatif (sous-performance)
        'beta': 1.5,
        'epsilon_std': 0.015
    },
    'CONSUMER_E': {
        'alpha': 0.0002,
        'beta': 0.9,
        'epsilon_std': 0.007
    }
}

# Génération des rendements
data = {'Date': pd.date_range(start='2022-01-01', periods=n_jours, freq='B')}  # B = Business days
data['MARKET'] = rendements_marche

for nom, params in actions.items():
    epsilon = np.random.normal(0, params['epsilon_std'], n_jours)
    rendements = params['alpha'] + params['beta'] * rendements_marche + epsilon
    data[nom] = rendements

# Création du DataFrame
df = pd.DataFrame(data)
df.set_index('Date', inplace=True)

print("📊 Dataset Créé")
print("="*60)
print(f"Période : {df.index[0].date()} à {df.index[-1].date()}")
print(f"Nombre de jours : {len(df)}")
print(f"Actions analysées : {', '.join([col for col in df.columns if col != 'MARKET'])}")
print("\nAperçu des données :")
print(df.head(10))

### 1.2 Vérification de la Qualité des Données

In [ ]:
# Vérification des valeurs manquantes
print("🔍 Contrôle Qualité des Données")
print("="*60)
print("\nValeurs manquantes :")
print(df.isnull().sum())

print("\nStatistiques de base :")
print(df.describe())

# Vérification d'outliers extrêmes
print("\n⚠️ Détection de rendements extrêmes (>5% ou <-5%) :")
for col in df.columns:
    extreme = df[(df[col] > 0.05) | (df[col] < -0.05)][col]
    if len(extreme) > 0:
        print(f"   {col}: {len(extreme)} jours avec rendements extrêmes")
    else:
        print(f"   {col}: Aucun rendement extrême détecté ✓")

---

## 📊 Étape 2 : Analyse Descriptive Complète

### 2.1 Statistiques Descriptives par Action

In [ ]:
# Calcul des statistiques pour chaque action
stats_df = pd.DataFrame()

actions_list = [col for col in df.columns if col != 'MARKET']

for action in actions_list:
    stats_df[action] = [
        df[action].mean(),                          # Moyenne quotidienne
        df[action].mean() * 252,                    # Rendement annualisé
        df[action].median(),                        # Médiane
        df[action].std(),                           # Volatilité quotidienne
        df[action].std() * np.sqrt(252),            # Volatilité annualisée
        stats.skew(df[action]),                     # Asymétrie
        stats.kurtosis(df[action]),                 # Aplatissement
        df[action].min(),                           # Rendement minimum
        df[action].max(),                           # Rendement maximum
        df[action].quantile(0.05),                  # VaR 5%
        df[action].quantile(0.95)                   # Percentile 95%
    ]

stats_df.index = ['Moyenne quotidienne', 'Rendement annualisé', 'Médiane', 
                  'Volatilité quotidienne', 'Volatilité annualisée',
                  'Skewness', 'Kurtosis (excès)', 'Min', 'Max', 'VaR 5%', 'Percentile 95%']

print("📊 STATISTIQUES DESCRIPTIVES COMPLÈTES")
print("="*80)
print(stats_df.to_string())

# Calcul du Sharpe Ratio (simplifié, sans taux sans risque)
print("\n💎 Ratio de Sharpe (simplifié, Rf=0) :")
print("="*60)
for action in actions_list:
    sharpe = (df[action].mean() * 252) / (df[action].std() * np.sqrt(252))
    print(f"   {action:12s} : {sharpe:.3f}")

### 2.2 Analyse de la Distribution des Rendements

In [ ]:
# Création d'un dashboard de distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, action in enumerate(actions_list):
    data = df[action]
    mean = data.mean()
    std = data.std()
    
    # Histogramme
    axes[idx].hist(data, bins=40, density=True, alpha=0.7, 
                   color='skyblue', edgecolor='black')
    
    # Courbe normale
    x = np.linspace(data.min(), data.max(), 100)
    axes[idx].plot(x, norm.pdf(x, mean, std), 'r-', linewidth=2, label='Normale')
    
    # Moyenne
    axes[idx].axvline(mean, color='green', linestyle='--', linewidth=1.5, 
                      label=f'μ={mean:.4%}')
    
    axes[idx].set_title(f'{action}\nSkew={stats.skew(data):.2f}, Kurt={stats.kurtosis(data):.2f}',
                        fontweight='bold')
    axes[idx].set_xlabel('Rendement quotidien')
    axes[idx].set_ylabel('Densité')
    axes[idx].legend(fontsize=8)
    axes[idx].grid(True, alpha=0.3)

# Supprimer le dernier subplot vide
fig.delaxes(axes[5])

plt.suptitle('📊 Distribution des Rendements par Action', 
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

---

## 📈 Étape 3 : Visualisations Professionnelles

### 3.1 Dashboard Multi-Panel

In [ ]:
# Création d'un dashboard professionnel 2x2
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Séries temporelles des rendements cumulés
ax1 = fig.add_subplot(gs[0, :])
rendements_cumules = (1 + df[actions_list]).cumprod()
for action in actions_list:
    ax1.plot(rendements_cumules.index, rendements_cumules[action], 
             linewidth=2, label=action, alpha=0.8)
ax1.set_title('📈 Performance Cumulée (Base 100)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Valeur du portefeuille')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)
ax1.axhline(1, color='black', linestyle='--', linewidth=1, alpha=0.5)

# 2. Boxplots comparatifs
ax2 = fig.add_subplot(gs[1, 0])
df[actions_list].boxplot(ax=ax2, patch_artist=True)
ax2.set_title('📦 Distribution Comparative (Boxplots)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Rendement quotidien')
ax2.grid(True, alpha=0.3, axis='y')
ax2.axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)

# 3. Heatmap de corrélation
ax3 = fig.add_subplot(gs[1, 1])
corr_matrix = df[actions_list].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, ax=ax3, cbar_kws={'label': 'Corrélation'})
ax3.set_title('🔥 Matrice de Corrélation', fontsize=14, fontweight='bold')

# 4. Rendement vs Risque (Scatter)
ax4 = fig.add_subplot(gs[2, 0])
for action in actions_list:
    rend_annuel = df[action].mean() * 252
    vol_annuelle = df[action].std() * np.sqrt(252)
    ax4.scatter(vol_annuelle, rend_annuel, s=200, alpha=0.7, label=action)
    ax4.annotate(action, (vol_annuelle, rend_annuel), 
                fontsize=10, ha='center', va='bottom')
ax4.set_xlabel('Volatilité Annualisée (Risque)', fontsize=11)
ax4.set_ylabel('Rendement Annualisé', fontsize=11)
ax4.set_title('💎 Rendement vs Risque', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)
ax4.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

# 5. Histogramme des rendements quotidiens du marché
ax5 = fig.add_subplot(gs[2, 1])
ax5.hist(df['MARKET'], bins=40, density=True, alpha=0.7, 
         color='gray', edgecolor='black', label='Marché')
x = np.linspace(df['MARKET'].min(), df['MARKET'].max(), 100)
ax5.plot(x, norm.pdf(x, df['MARKET'].mean(), df['MARKET'].std()), 
         'r-', linewidth=2, label='Normale')
ax5.axvline(df['MARKET'].mean(), color='green', linestyle='--', linewidth=2)
ax5.set_title('📊 Distribution du Marché', fontsize=14, fontweight='bold')
ax5.set_xlabel('Rendement quotidien')
ax5.set_ylabel('Densité')
ax5.legend()
ax5.grid(True, alpha=0.3)

plt.suptitle('🚀 DASHBOARD ANALYTIQUE COMPLET', 
             fontsize=18, fontweight='bold', y=0.995)
plt.show()

---

## 🧪 Étape 4 : Tests Statistiques

### 4.1 Tests de Normalité

In [ ]:
print("🧪 TESTS DE NORMALITÉ")
print("="*80)
print("\nH₀ : Les rendements suivent une distribution normale")
print("H₁ : Les rendements ne suivent PAS une distribution normale\n")

results_normalite = []

for action in actions_list:
    # Test de Shapiro-Wilk
    shapiro_stat, shapiro_p = stats.shapiro(df[action])
    
    # Test de Kolmogorov-Smirnov
    ks_stat, ks_p = stats.kstest(df[action], 'norm', 
                                  args=(df[action].mean(), df[action].std()))
    
    results_normalite.append({
        'Action': action,
        'Shapiro p-value': shapiro_p,
        'Shapiro décision': 'Rejeter H₀' if shapiro_p < 0.05 else 'Ne pas rejeter H₀',
        'KS p-value': ks_p,
        'KS décision': 'Rejeter H₀' if ks_p < 0.05 else 'Ne pas rejeter H₀'
    })

df_normalite = pd.DataFrame(results_normalite)
print(df_normalite.to_string(index=False))

print("\n💡 Interprétation :")
print("   Si p-value < 0.05 → Rejeter H₀ (distribution NON normale)")
print("   Si p-value ≥ 0.05 → Ne pas rejeter H₀ (distribution potentiellement normale)")

### 4.2 Tests de Rendement Moyen > 0

In [ ]:
print("\n🧪 TESTS : RENDEMENT MOYEN SIGNIFICATIVEMENT > 0 ?")
print("="*80)
print("H₀ : μ = 0 (pas de rendement positif)")
print("H₁ : μ > 0 (rendement positif significatif)\n")

results_ttest = []

for action in actions_list:
    # T-test à un échantillon (test unilatéral)
    t_stat, p_value_bilateral = stats.ttest_1samp(df[action], 0)
    p_value_unilateral = p_value_bilateral / 2 if t_stat > 0 else 1 - p_value_bilateral / 2
    
    rend_moyen = df[action].mean()
    rend_annuel = rend_moyen * 252
    
    results_ttest.append({
        'Action': action,
        'Rendement moyen': f"{rend_moyen:.6f}",
        'Rendement annuel': f"{rend_annuel:.4%}",
        'T-statistic': f"{t_stat:.4f}",
        'P-value': f"{p_value_unilateral:.6f}",
        'Décision (α=0.05)': '✅ Rendement > 0' if p_value_unilateral < 0.05 else '❌ Pas significatif'
    })

df_ttest = pd.DataFrame(results_ttest)
print(df_ttest.to_string(index=False))

### 4.3 Comparaison des Performances (ANOVA)

In [ ]:
print("\n🧪 TEST ANOVA : Les actions ont-elles des rendements moyens différents ?")
print("="*80)
print("H₀ : Toutes les actions ont le même rendement moyen")
print("H₁ : Au moins une action a un rendement moyen différent\n")

# ANOVA à un facteur
f_stat, p_value = stats.f_oneway(*[df[action] for action in actions_list])

print(f"F-statistic : {f_stat:.4f}")
print(f"P-value     : {p_value:.6f}")
print(f"\nDécision (α=0.05) : {'Rejeter H₀' if p_value < 0.05 else 'Ne pas rejeter H₀'}")

if p_value < 0.05:
    print("\n✅ Il existe des différences significatives de performance entre les actions")
else:
    print("\n❌ Pas de différence significative détectée entre les actions")

---

## 📏 Étape 5 : Intervalles de Confiance

### 5.1 IC pour les Rendements Moyens

In [ ]:
print("📏 INTERVALLES DE CONFIANCE À 95% POUR RENDEMENTS ANNUALISÉS")
print("="*80)

results_ic = []

for action in actions_list:
    n = len(df[action])
    mean = df[action].mean()
    std_err = stats.sem(df[action])  # Erreur standard
    
    # IC avec distribution t
    ci = stats.t.interval(0.95, n-1, loc=mean, scale=std_err)
    
    # Annualisation
    ci_annuel = (ci[0] * 252, ci[1] * 252)
    mean_annuel = mean * 252
    largeur_ic = ci_annuel[1] - ci_annuel[0]
    
    results_ic.append({
        'Action': action,
        'Rendement annuel': f"{mean_annuel:.2%}",
        'IC 95% - Borne inf': f"{ci_annuel[0]:.2%}",
        'IC 95% - Borne sup': f"{ci_annuel[1]:.2%}",
        'Largeur IC': f"{largeur_ic:.2%}"
    })

df_ic = pd.DataFrame(results_ic)
print(df_ic.to_string(index=False))

print("\n💡 Interprétation :")
print("   Nous sommes confiants à 95% que le vrai rendement annuel se trouve dans cet intervalle")

### 5.2 Visualisation des Intervalles de Confiance

In [ ]:
# Graphique des IC
fig, ax = plt.subplots(figsize=(12, 6))

positions = np.arange(len(actions_list))

for idx, action in enumerate(actions_list):
    n = len(df[action])
    mean = df[action].mean() * 252
    std_err = stats.sem(df[action]) * 252
    ci = stats.t.interval(0.95, n-1, loc=mean, scale=std_err)
    
    # Point estimate
    ax.plot(idx, mean, 'o', markersize=10, label=action)
    
    # Intervalle de confiance
    ax.errorbar(idx, mean, yerr=[[mean - ci[0]], [ci[1] - mean]], 
                fmt='o', capsize=5, capthick=2, linewidth=2)

ax.axhline(0, color='red', linestyle='--', linewidth=1.5, alpha=0.5, label='Rendement nul')
ax.set_xticks(positions)
ax.set_xticklabels(actions_list)
ax.set_ylabel('Rendement Annualisé', fontsize=12)
ax.set_title('📏 Intervalles de Confiance à 95% pour les Rendements Annualisés', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.show()

---

## 📈 Étape 6 : Modèle CAPM et Régression Linéaire

### 6.1 Calcul des Betas et Alphas

In [ ]:
print("📈 MODÈLE CAPM : CALCUL DES BETAS ET ALPHAS")
print("="*80)
print("Modèle : R_action = α + β × R_marché + ε\n")

results_capm = []

for action in actions_list:
    # Régression linéaire
    X = sm.add_constant(df['MARKET'])  # Ajout de la constante pour α
    y = df[action]
    
    model = sm.OLS(y, X).fit()
    
    alpha = model.params['const']
    beta = model.params['MARKET']
    r_squared = model.rsquared
    p_value_beta = model.pvalues['MARKET']
    
    # Annualisation de l'alpha
    alpha_annuel = alpha * 252
    
    results_capm.append({
        'Action': action,
        'Alpha (quotidien)': f"{alpha:.6f}",
        'Alpha annualisé': f"{alpha_annuel:.4%}",
        'Beta': f"{beta:.4f}",
        'R²': f"{r_squared:.4f}",
        'P-value (β)': f"{p_value_beta:.6f}",
        'β Significatif': '✅' if p_value_beta < 0.05 else '❌'
    })

df_capm = pd.DataFrame(results_capm)
print(df_capm.to_string(index=False))

print("\n💡 Interprétation du Beta :")
print("   β = 1.0  → Action suit le marché")
print("   β > 1.0  → Action plus volatile que le marché (aggressive)")
print("   β < 1.0  → Action moins volatile que le marché (défensive)")
print("\n💡 Interprétation de l'Alpha :")
print("   α > 0    → Performance excédentaire (sur-performance)")
print("   α < 0    → Sous-performance par rapport au marché")

### 6.2 Visualisation des Régressions CAPM

In [ ]:
# Scatter plots avec droites de régression
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, action in enumerate(actions_list):
    X = df['MARKET']
    y = df[action]
    
    # Régression
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)
    
    # Scatter
    axes[idx].scatter(X, y, alpha=0.5, s=20, color='steelblue')
    
    # Droite de régression
    x_fit = np.linspace(X.min(), X.max(), 100)
    y_fit = intercept + slope * x_fit
    axes[idx].plot(x_fit, y_fit, 'r-', linewidth=2.5, 
                   label=f'β={slope:.2f}, α={intercept:.5f}')
    
    axes[idx].set_xlabel('Rendement Marché', fontsize=10)
    axes[idx].set_ylabel(f'Rendement {action}', fontsize=10)
    axes[idx].set_title(f'{action}\nR²={r_value**2:.3f}', fontweight='bold')
    axes[idx].legend(fontsize=9)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].axhline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
    axes[idx].axvline(0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)

# Supprimer le dernier subplot
fig.delaxes(axes[5])

plt.suptitle('📈 Modèle CAPM : Régressions Action vs Marché', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### 6.3 Analyse des Résidus (Validation du Modèle)

In [ ]:
# Analyse des résidus pour TECH_A (exemple)
action_test = 'TECH_A'

X = sm.add_constant(df['MARKET'])
y = df[action_test]
model = sm.OLS(y, X).fit()

# Résidus
residus = model.resid
fitted = model.fittedvalues

# Tests sur les résidus
print(f"🔍 ANALYSE DES RÉSIDUS POUR {action_test}")
print("="*80)

# 1. Test de normalité des résidus
shapiro_stat, shapiro_p = stats.shapiro(residus)
print(f"\n1. Test de Normalité des Résidus (Shapiro-Wilk) :")
print(f"   P-value : {shapiro_p:.6f}")
print(f"   Décision: {'Résidus normaux ✓' if shapiro_p > 0.05 else 'Résidus NON normaux ✗'}")

# 2. Autocorrélation (Durbin-Watson)
from statsmodels.stats.stattools import durbin_watson
dw = durbin_watson(residus)
print(f"\n2. Test de Durbin-Watson (autocorrélation) :")
print(f"   Statistique DW : {dw:.4f}")
print(f"   Interprétation : ", end="")
if 1.5 < dw < 2.5:
    print("Pas d'autocorrélation significative ✓")
elif dw < 1.5:
    print("Autocorrélation positive détectée ✗")
else:
    print("Autocorrélation négative détectée ✗")

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Résidus vs Fitted
axes[0, 0].scatter(fitted, residus, alpha=0.5, s=20)
axes[0, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Valeurs Prédites')
axes[0, 0].set_ylabel('Résidus')
axes[0, 0].set_title('Résidus vs Valeurs Prédites', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Histogramme des résidus
axes[0, 1].hist(residus, bins=30, density=True, alpha=0.7, edgecolor='black')
x = np.linspace(residus.min(), residus.max(), 100)
axes[0, 1].plot(x, norm.pdf(x, np.mean(residus), np.std(residus)), 'r-', linewidth=2)
axes[0, 1].set_xlabel('Résidus')
axes[0, 1].set_ylabel('Densité')
axes[0, 1].set_title('Distribution des Résidus', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Q-Q Plot
stats.probplot(residus, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot des Résidus', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Résidus dans le temps
axes[1, 1].plot(residus, linewidth=1, alpha=0.7)
axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=1.5)
axes[1, 1].axhline(np.std(residus), color='orange', linestyle=':', linewidth=1)
axes[1, 1].axhline(-np.std(residus), color='orange', linestyle=':', linewidth=1)
axes[1, 1].set_xlabel('Observation')
axes[1, 1].set_ylabel('Résidu')
axes[1, 1].set_title('Résidus dans le Temps', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f'🔍 Diagnostic des Résidus - {action_test}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 📑 Étape 7 : Rapport Final et Recommandations

### 7.1 Synthèse des Résultats

In [ ]:
# Création d'un tableau de synthèse
synthese = pd.DataFrame()

for action in actions_list:
    # Statistiques de base
    rend_annuel = df[action].mean() * 252
    vol_annuelle = df[action].std() * np.sqrt(252)
    sharpe = rend_annuel / vol_annuelle
    
    # CAPM
    X = sm.add_constant(df['MARKET'])
    model = sm.OLS(df[action], X).fit()
    alpha = model.params['const'] * 252
    beta = model.params['MARKET']
    r2 = model.rsquared
    
    # Skewness & Kurtosis
    skew = stats.skew(df[action])
    kurt = stats.kurtosis(df[action])
    
    synthese[action] = [
        f"{rend_annuel:.2%}",
        f"{vol_annuelle:.2%}",
        f"{sharpe:.3f}",
        f"{alpha:.2%}",
        f"{beta:.3f}",
        f"{r2:.3f}",
        f"{skew:.3f}",
        f"{kurt:.3f}"
    ]

synthese.index = ['Rendement annuel', 'Volatilité annuelle', 'Sharpe Ratio', 
                  'Alpha (CAPM)', 'Beta (CAPM)', 'R² (CAPM)', 
                  'Skewness', 'Kurtosis']

print("📊 TABLEAU DE SYNTHÈSE FINALE")
print("="*100)
print(synthese.to_string())

### 7.2 Recommandations d'Investissement

In [ ]:
print("\n\n💼 RECOMMANDATIONS D'INVESTISSEMENT")
print("="*100)

# Classement par Sharpe Ratio
sharpe_ranking = {}
for action in actions_list:
    rend = df[action].mean() * 252
    vol = df[action].std() * np.sqrt(252)
    sharpe_ranking[action] = rend / vol

sorted_sharpe = sorted(sharpe_ranking.items(), key=lambda x: x[1], reverse=True)

print("\n🏆 CLASSEMENT PAR SHARPE RATIO (Rendement ajusté du risque) :")
print("-" * 60)
for rank, (action, sharpe) in enumerate(sorted_sharpe, 1):
    print(f"   {rank}. {action:12s} : Sharpe = {sharpe:.3f}")

# Analyse par profil d'investisseur
print("\n\n📈 RECOMMANDATIONS PAR PROFIL D'INVESTISSEUR :")
print("-" * 80)

# Profil Conservateur
print("\n🛡️  PROFIL CONSERVATEUR (faible risque) :")
vol_dict = {action: df[action].std() * np.sqrt(252) for action in actions_list}
conservateur = min(vol_dict, key=vol_dict.get)
print(f"   ➤ Recommandation : {conservateur}")
print(f"   ➤ Volatilité : {vol_dict[conservateur]:.2%}")
print(f"   ➤ Beta : {sm.OLS(df[conservateur], sm.add_constant(df['MARKET'])).fit().params['MARKET']:.3f}")

# Profil Équilibré
print("\n⚖️  PROFIL ÉQUILIBRÉ (rendement/risque optimal) :")
equilibre = sorted_sharpe[0][0]
print(f"   ➤ Recommandation : {equilibre}")
print(f"   ➤ Sharpe Ratio : {sorted_sharpe[0][1]:.3f}")
print(f"   ➤ Rendement : {df[equilibre].mean() * 252:.2%}")
print(f"   ➤ Volatilité : {df[equilibre].std() * np.sqrt(252):.2%}")

# Profil Agressif
print("\n🚀 PROFIL AGRESSIF (rendement maximal) :")
rend_dict = {action: df[action].mean() * 252 for action in actions_list}
agressif = max(rend_dict, key=rend_dict.get)
print(f"   ➤ Recommandation : {agressif}")
print(f"   ➤ Rendement annuel : {rend_dict[agressif]:.2%}")
print(f"   ➤ Beta : {sm.OLS(df[agressif], sm.add_constant(df['MARKET'])).fit().params['MARKET']:.3f}")

# Actions à éviter
print("\n\n⚠️  ACTIONS À ÉVITER OU SURVEILLER :")
print("-" * 60)
for action in actions_list:
    alpha = sm.OLS(df[action], sm.add_constant(df['MARKET'])).fit().params['const'] * 252
    if alpha < 0:
        print(f"   ⚠️  {action} : Alpha négatif ({alpha:.2%}) - Sous-performance systématique")

# Diversification
print("\n\n🧩 RECOMMANDATIONS DE DIVERSIFICATION :")
print("-" * 80)
corr_matrix = df[actions_list].corr()
print("\nPaires d'actions faiblement corrélées (bonnes pour diversification) :")
for i in range(len(actions_list)):
    for j in range(i+1, len(actions_list)):
        corr = corr_matrix.iloc[i, j]
        if corr < 0.5:
            print(f"   ✅ {actions_list[i]} & {actions_list[j]} : Corrélation = {corr:.3f}")

print("\n" + "="*100)
print("\n✅ ANALYSE STATISTIQUE TERMINÉE")
print("\n📊 Ce rapport fournit une base solide pour des décisions d'investissement éclairées.")
print("⚠️  Rappel : Les performances passées ne garantissent pas les performances futures.")

---

## 🎓 Conclusion du Projet

### ✅ Compétences Démontrées

Dans ce projet, vous avez appliqué avec succès :

1. **📊 Statistiques Descriptives**
   - Calcul et interprétation de toutes les mesures de tendance centrale et dispersion
   - Analyse de distributions (skewness, kurtosis)
   - Détection d'anomalies statistiques

2. **📈 Visualisation de Données**
   - Création de dashboards professionnels multi-panels
   - Heatmaps de corrélation
   - Graphiques de performance temporelle

3. **🧪 Tests d'Hypothèses**
   - Tests de normalité (Shapiro-Wilk, Kolmogorov-Smirnov)
   - Tests t pour moyennes
   - ANOVA pour comparaisons multiples

4. **📏 Intervalles de Confiance**
   - Calcul d'IC pour rendements et volatilités
   - Visualisation des incertitudes

5. **📈 Régression et Modélisation**
   - Modèle CAPM complet (alpha, beta)
   - Analyse de la qualité du modèle (R²)
   - Diagnostic des résidus

6. **💼 Applications Finance**
   - Ratio de Sharpe
   - Analyse rendement/risque
   - Recommandations d'investissement

### 🚀 Prochaines Étapes

Vous êtes maintenant prêt pour :
- **Probabilités avancées** (Chapitre 09)
- **Machine Learning** avec bases statistiques solides
- **Finance quantitative** professionnelle

---

## 📚 Ressources Complémentaires

- 📖 [Cours complet](../../cours/Phase_1_Mathematiques/Maths_08_Statistiques.ipynb)
- 📝 [Exercices pratiques](exercices_08_statistiques.ipynb)
- ✅ [Solutions détaillées](solutions_08_statistiques.ipynb)

---

**Félicitations pour avoir complété ce projet ! 🎉**